In [2]:
import os
import sys

# Allowing Jupyter to load modules and clases from the file system
module_path = os.path.abspath(os.path.join("../src"))
if module_path not in sys.path:
    sys.path.append(module_path)

### Window Sentence Retrieval

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
import utils

import os
import openai

openai.api_key = utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [6]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["../data/eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [7]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.core.schema.Document'>
Doc ID: d99a7be2-66e7-4d75-9ffb-c8e7dc3ce531
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


In [9]:
from llama_index.core import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

### Window Sentence Retrieval Set-Up

In [11]:
from llama_index.core.node_parser import SentenceWindowNodeParser

# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [12]:
text = "hello. how are you? I am fine!  "

nodes = node_parser.get_nodes_from_documents([Document(text=text)])

In [13]:
print([x.text for x in nodes])

['hello. ', 'how are you? ', 'I am fine!  ']


In [14]:
print(nodes[1].metadata["window"])

hello.  how are you?  I am fine!  


In [15]:
text = "hello. foo bar. cat dog. mouse"

nodes = node_parser.get_nodes_from_documents([Document(text=text)])

In [16]:
print([x.text for x in nodes])

['hello. ', 'foo bar. ', 'cat dog. ', 'mouse']


In [17]:
print(nodes[0].metadata["window"])

hello.  foo bar.  cat dog.  mouse


#### Building the index

In [18]:
# from llama_index.llms import OpenAI
from llama_index.llms.azure_openai import AzureOpenAI

# llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
llm = AzureOpenAI(
    model="gpt-4o",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_API_ENDPOINT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_deployment=os.getenv("AZURE_OPENAI_API_LLM_DEPLOYMENT_NAME"),
    temperature=0.1,
)

In [19]:
from llama_index import ServiceContext

sentence_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    # embed_model="local:BAAI/bge-large-en-v1.5"
    node_parser=node_parser,
)

ImportError: cannot import name 'ServiceContext' from 'llama_index' (unknown location)